In [1]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/CS475/'

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install trl
!pip install unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 26.3 MB/s eta 0:00:00

In [3]:
from huggingface_hub import login
login()

In [4]:
import transformers
import torch
from trl import DPOConfig, DPOTrainer
from transformers import pipeline, AutoTokenizer
from transformers import TrainingArguments, TextStreamer
from datasets import load_dataset
from transformers.pipelines.pt_utils import KeyDataset
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported

from collections import Counter

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("reggans/Llama-3.2-3B-Instruct-bnb-4bit-DPO-All")
model = AutoModelForCausalLM.from_pretrained("reggans/Llama-3.2-3B-Instruct-bnb-4bit-DPO-All")

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:  20%|#9        | 440M/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

# Load & Clean DS

In [ ]:
ds = load_dataset("amphora/mprom-cultureatlas",
                  split='train')
ds

In [6]:
countries = KeyDataset(ds, 'country')
Counter(countries).most_common()

[('Afghanistan', 466),
 ('China', 426),
 ('Canada', 344),
 ("Democratic People's Republic of Korea", 224),
 ('Denmark', 219),
 ('Brazil', 199),
 ('Egypt', 129),
 ('Algeria', 92),
 ('Cambodia', 91),
 ('Albania', 86),
 ('Argentina', 85),
 ('Estonia', 80),
 ('Bangladesh', 77),
 ('Bhutan', 71),
 ('Angola', 56),
 ('Bosnia and Herzegovina', 49),
 ('Ecuador', 42),
 ('Bulgaria', 41),
 ('Andorra', 38),
 ('Belize', 37),
 ('Djibouti', 33),
 ('Cameroon', 31),
 ('El Salvador', 29),
 ('Belarus', 21),
 ('Botswana', 20),
 ('Belgium', 16),
 ('Dominican Republic', 15),
 ('Eritrea', 14),
 ('Burundi', 10),
 ('Chad', 10),
 ('Plurinational State of Bolivia', 10),
 ('Benin', 9),
 ('Dominica', 9),
 ('Burkina Faso', 8),
 ('Eswatini', 8),
 ('Bahamas', 6),
 ('Chile', 5),
 ('Barbados', 4),
 ('Antigua and Barbuda', 3),
 ('Bahrain', 3),
 ('Central African Republic', 3)]

In [7]:
chosen = ['Afghanistan', 'China', 'Egypt', 'Cambodia']

In [8]:
cleaned_ds = ds.filter(lambda x: x['country'] in chosen)
cleaned_ds = cleaned_ds.remove_columns(['country', 'keyword'])
cleaned_ds = cleaned_ds.rename_column('instruction', 'prompt')
cleaned_ds = cleaned_ds.rename_column('positive_sample', 'chosen')
cleaned_ds = cleaned_ds.rename_column('negative_sample', 'rejected')
cleaned_ds

Filter:   0%|          | 0/3119 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1112
})

# Load Model

In [ ]:
max_seq_length = 5020
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
model.print_trainable_parameters()

FileNotFoundError: unsloth/Llama-3.2-8B-Instruct-bnb-4bit/*.json (repository not found)

# Finetune

### All Countries

In [ ]:
run_name = "Llama-3.2-3B-Instruct-bnb-4bit-DPO-All"
train_args = DPOConfig(output_dir=root + run_name,
                       logging_steps=10,
                       num_train_epochs=10,
                       fp16=not is_bfloat16_supported(),
                       bf16=is_bfloat16_supported(),
                       optim="adamw_8bit",
                       per_device_train_batch_size=4,
                       run_name=run_name,
                       torch_empty_cache_steps=10)
trainer = DPOTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=cleaned_ds,
    args=train_args
)
trainer.train()

In [ ]:
model.push_to_hub('reggans/' + run_name)
tokenizer.push_to_hub('reggans/' + run_name)

### Per Country

In [9]:
for country in chosen:
  torch.cuda.empty_cache()
  print(f'Finetuning for {country}')

  # Load model
  max_seq_length = 5020
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
      max_seq_length=max_seq_length,
      load_in_4bit=True,
      dtype=None,
  )
  model = FastLanguageModel.get_peft_model(
      model,
      r=16,
      lora_alpha=16,
      lora_dropout=0.1,
      target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
      use_rslora=True,
      use_gradient_checkpointing="unsloth",
      random_state = 32,
      loftq_config = None,
  )

  # Filter and clean DS
  country_ds = ds.filter(lambda x: x['country'] == country)
  country_ds = country_ds.remove_columns(['country', 'keyword'])
  country_ds = country_ds.rename_column('instruction', 'prompt')
  country_ds = country_ds.rename_column('positive_sample', 'chosen')
  country_ds = country_ds.rename_column('negative_sample', 'rejected')

  # Run training
  run_name = f"Llama-3.2-3B-Instruct-bnb-4bit-DPO-{country}"
  train_args = DPOConfig(output_dir=root + run_name,
                        logging_steps=10,
                        num_train_epochs=10,
                        fp16=not is_bfloat16_supported(),
                        bf16=is_bfloat16_supported(),
                        optim="adamw_8bit",
                        per_device_train_batch_size=4,
                        run_name=run_name,
                        torch_empty_cache_steps=100)
  trainer = DPOTrainer(
      model=model,
      processing_class=tokenizer,
      train_dataset=country_ds,
      args=train_args
  )
  trainer.train()

  # Save model
  model.push_to_hub('reggans/' + run_name)
  tokenizer.push_to_hub('reggans/' + run_name)

Finetuning for Afghanistan
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

KeyboardInterrupt: 